In [1]:
import argparse
import copy

import matplotlib.pyplot as plt
import tensorflow as tf
import c3.libraries.algorithms as algorithms
import c3.libraries.fidelities as fidelities
import c3.signal.gates as gates
from c3.c3objs import Quantity as Qty
from c3.experiment import Experiment as Exp
from c3.generator.generator import Generator as Gnr
from c3.libraries import constants
from c3.model import Model as Mdl
from c3.parametermap import ParameterMap as PMap
from c3.utils import qt_utils
from utils import *

2021-10-27 18:11:39.870687: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-27 18:11:39.870719: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#parser = argparse.ArgumentParser()
#parser.add_argument("output", help="Output directory")
#args = parser.parse_args()
#output_dir = args.output
output_dir = "./output_1"

In [3]:
# Qubits and coupling
qubit_lvls = [4, 4]
numActiveLevels = 4
frequencies = [5e9, 4e9]
anharmonicities = [-300e6, -300e6]
t1 = [27e-6, 27e-6]
t2star = [39e-6, 39e-6]
qubit_temp = [50e-3, 50e-3]

qubits = []
for i in range(len(qubit_lvls)):
    qubits.append(chip.Qubit(
        name=f"Q{i + 1}",
        desc=f"Qubit {i + 1}",
        freq=Qty(
            value=frequencies[i],
            min_val=frequencies[i] - 5e6,
            max_val=frequencies[i] + 5e6,
            unit='Hz 2pi'
        ),
        anhar=Qty(
            value=anharmonicities[i],
            min_val=-380e6,
            max_val=-120e6,
            unit='Hz 2pi'
        ),
        hilbert_dim=qubit_lvls[i],
        t1=Qty(
            value=t1[i],
            min_val=1e-6,
            max_val=90e-6,
            unit='s'
        ),
        t2star=Qty(
            value=t2star[i],
            min_val=10e-6,
            max_val=90e-3,
            unit='s'
        ),
        temp=Qty(
            value=qubit_temp[i],
            min_val=0.0,
            max_val=0.12,
            unit='K'
        )
    ))
print(qubits)

#couplings = []
#for i in range(len(qubits)-1):
#    couplings.append(createCoupling(qubits[i], qubits[i+1], 0))

[<c3.libraries.chip.Qubit object at 0x7f6c14bd8670>, <c3.libraries.chip.Qubit object at 0x7f6c14bd88b0>]


2021-10-27 18:11:44.413415: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-27 18:11:44.413816: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-27 18:11:44.413859: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-27 18:11:44.413924: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (localhost.localdomain): /proc/driver/nvidia/version does not exist
2021-10-27 18:11:44.418289: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


In [4]:
# Drive hamiltonians
drives = []
for i in range(len(qubits)):
    drives.append(chip.Drive(
        name=f"d{i+1}",
        desc=f"Drive {i+1}",
        comment=f"Drive line on qubit {qubits[i].name}",
        connected=[qubits[i].name],
        hamiltonian_func=hamiltonians.x_drive
    ))

In [5]:
# Set up the model
model = Mdl(qubits, drives)# + couplings)
model.set_lindbladian(False)
model.set_dressed(True)

In [6]:
# Generator
sim_res = 100e9  # Resolution for numerical simulation
awg_res = 2e9  # Realistic, limited resolution of an AWG
chain = ["LO", "AWG", "DigitalToAnalog", "Response", "Mixer", "VoltsToHertz"]
chains = {f"{d.name}": chain for d in drives}
generator = Gnr(
    devices={
        "LO": devices.LO(name='lo', resolution=sim_res, outputs=1),
        "AWG": devices.AWG(name='awg', resolution=awg_res, outputs=1),
        "DigitalToAnalog": devices.DigitalToAnalog(
            name="dac",
            resolution=sim_res,
            inputs=1,
            outputs=1
        ),
        "Response": devices.Response(
            name='resp',
            rise_time=Qty(
                value=0.3e-9,
                min_val=0.05e-9,
                max_val=0.6e-9,
                unit='s'
            ),
            resolution=sim_res,
            inputs=1,
            outputs=1
        ),
        "Mixer": devices.Mixer(name='mixer', inputs=2, outputs=1),
        "VoltsToHertz": devices.VoltsToHertz(
            name='v_to_hz',
            V_to_Hz=Qty(
                value=1e9,
                min_val=0.9e9,
                max_val=1.1e9,
                unit='Hz/V'
            ),
            inputs=1,
            outputs=1
        )
    },
    chains=chains,
)

In [10]:
# Gate
t_final = 15e-9   # Time for single qubit gates
sideband = 50e6
sigma = t_final / 15
sigma2 = sigma / 2
relative_amp = 2
pwc_times = tf.linspace(0.0, t_final, 30)

pwc_values = tf.exp(-((pwc_times - t_final / 2) ** 2) / (2 * sigma ** 2)) - tf.exp(
    -((pwc_times - t_final / 2) ** 2) / (2 * sigma2 ** 2)) * relative_amp
pwc_values2 = 0.1 * tf.exp(-((pwc_times - t_final / 2) ** 2) / (2 * sigma ** 2))
pwc_env = pulse.Envelope(
    name="pwc1",
    desc="PWC envelope",
    params={
        "amp": Qty(value=0.5, min_val=0.2, max_val=0.6, unit="V"),
        "t_final": Qty(value=t_final, min_val=0.9 * t_final, max_val=1.1 * t_final, unit="s"),
        "xy_angle": Qty(value=0.0, min_val=-1.5 * np.pi, max_val=2.5 * np.pi, unit="rad"),
        "freq_offset": Qty(value=-53e6, min_val=-56e6, max_val=-52e6, unit="Hz 2pi"),
        "delta": Qty(value=-1, min_val=-5, max_val=5, unit=""),
        "t_bin_start": Qty(0),
        "t_bin_end": Qty(t_final),
        "inphase": Qty(pwc_values),
    },
    shape=envelopes.pwc_shape,
)
pwc_env2 = pulse.Envelope(
    name="pwc2",
    desc="PWC envelope",
    params={
        "amp": Qty(value=0.5, min_val=0.2, max_val=0.6, unit="V"),
        "t_final": Qty(value=t_final, min_val=0.9 * t_final, max_val=1.1 * t_final, unit="s"),
        "xy_angle": Qty(value=0.0, min_val=-1.5 * np.pi, max_val=2.5 * np.pi, unit="rad"),
        "freq_offset": Qty(value=-53e6, min_val=-56e6, max_val=-52e6, unit="Hz 2pi"),
        "delta": Qty(value=-1, min_val=-5, max_val=5, unit=""),
        "t_bin_start": Qty(0),
        "t_bin_end": Qty(t_final),
        "inphase": Qty(pwc_values2),
    },
    shape=envelopes.pwc_shape,
)
lo_frequencies = [4.7e9, 3.7e9]
carriers = []
for i in range(len(lo_frequencies)):
    carriers.append(pulse.Carrier(
        name="carrier",
        desc="Frequency of the local oscillator",
        params={
            'freq': Qty(
                value=lo_frequencies[i],
                min_val=0.75 * lo_frequencies[i],
                max_val=1.25 * lo_frequencies[i],
                unit='Hz 2pi'
            ),
            'framechange': Qty(
                value=0.0,
                min_val=-np.pi,
                max_val=3 * np.pi,
                unit='rad'
            )
        }
    ))

In [11]:
# Instruction
idealGate = qt_utils.np_kron_n([constants.Id, constants.x90p])
rx90p_q1 = gates.Instruction(
    name="rx90p", targets=[0], t_start=0.0, t_end=t_final, channels=[d.name for d in drives],
    ideal=idealGate
)
rx90p_q1.add_component(pwc_env, "d1")
rx90p_q1.add_component(carriers[0], "d1")
rx90p_q1.add_component(pwc_env2, "d2")
rx90p_q1.add_component(carriers[1], "d2")
#rx90p_q1.comps["d2"]["carrier"].params["framechange"].set_value(
#    (-sideband * t_final) * 2 * np.pi % (2 * np.pi)
#)

perfect_gate = rx90p_q1.get_ideal_gate(
    [q.hilbert_dim for q in qubits],
    [q.index for q in qubits],
    active_levels=numActiveLevels
)
plotMatrix(perfect_gate, output_dir + "/ideal_gate.png",
           output_dir + "/ideal_gate_phase.png",
           output_dir + "/ideal_gate_abs.png")

saving plot in ./output_1/ideal_gate.png
saving plot in ./output_1/ideal_gate_phase.png
saving plot in ./output_1/ideal_gate_abs.png


In [12]:
# Set up experiment
parameter_map = PMap(instructions=[rx90p_q1], model=model, generator=generator)
exp = Exp(pmap=parameter_map)
exp.set_opt_gates(['rx90p[0]'])
unitaries = exp.compute_propagators()

psi_init = [[0] * model.tot_dim]
psi_init[0][0] = 1
init_state = tf.transpose(tf.constant(psi_init, tf.complex128))

In [13]:
def plot_dynamics(exp: Experiment, psi_init, seq, filename=None):
    model = exp.pmap.model
    dUs = exp.partial_propagators
    psi_t = psi_init.numpy()
    pop_t = exp.populations(psi_t, model.lindbladian)
    for gate in seq:
        for du in dUs[gate]:
            psi_t = np.matmul(du.numpy(), psi_t)
            pops = exp.populations(psi_t, model.lindbladian)
            pop_t = np.append(pop_t, pops, axis=1)

    fig, axs = plt.subplots(1, 1)
    ts = exp.ts
    dt = ts[1] - ts[0]
    ts = np.linspace(0.0, dt * pop_t.shape[1], pop_t.shape[1])
    axs.plot(ts / 1e-9, pop_t.T)
    axs.grid(linestyle="--")
    axs.tick_params(
        direction="in", left=True, right=True, top=True, bottom=True
    )
    axs.set_xlabel('Time [ns]')
    axs.set_ylabel('Population')
    plt.legend(model.state_labels, ncol=2, bbox_to_anchor=(1.1, 1.05))
    plt.tight_layout()

    if filename is not None:
        plt.savefig(filename)
        plt.close()
    pass


def getOutputFromDevice(gen: Generator, gate: Instruction, channel: str, deviceName: str):
    gen.generate_signals(gate)
    return gen.getDeviceOutput(channel, deviceName)


def getEnvelope(gen: Generator, gate: Instruction, channel: str):
    full_signal = gen.generate_signals(gate)[channel]
    values1 = full_signal["values"].numpy()
    envelope = getOutputFromDevice(gen, gate, channel, "Response")
    values2 = envelope["inphase"].numpy()
    factor = np.max(np.abs(values1)) / np.max(np.abs(values2))
    return envelope["ts"].numpy(), factor * envelope["inphase"].numpy(), factor * envelope["quadrature"].numpy()

In [14]:
sequence = ['rx90p[0]']
for drive in drives:
    signal = generator.generate_signals(rx90p_q1)[drive.name]
    plotSignalWithEnvelope(signal["ts"].numpy(), signal["values"].numpy(),
                           getEnvelope(generator, rx90p_q1, drive.name),
                           pwc_times,
                           output_dir + f"/signal_before_{drive.name}.png",
                           spectrum_cut=1e-4)

plot_dynamics(exp, init_state, sequence, output_dir + "/population_before.png")
population = runTimeEvolutionDefault(exp, init_state, sequence)
if len(qubits) > 1:
    plotSplittedOccupations(exp, population, sequence, output_dir + "/population_before_splitted.png")
plotMatrix(
    exp.propagators[rx90p_q1.get_key()],
    output_dir + "/propagator_before.png",
    output_dir + "/propagator_before_phase.png",
    output_dir + "/propagator_before_abs.png",
    labels=[str(x) for x in model.state_labels]
)

saving plot in ./output_1/signal_before_d1.png
saving plot in ./output_1/signal_before_d2.png
saving plot in ./output_1/population_before_splitted.png
saving plot in ./output_1/propagator_before.png
saving plot in ./output_1/propagator_before_phase.png
saving plot in ./output_1/propagator_before_abs.png


In [15]:
# Optimise
generator.devices['AWG'].enable_drag_2()
opt_gates = ["rx90p[0]"]
gateset_opt_map = []
for i in range(1, len(drives)+1):
    drive = drives[i-1]
    gateset_opt_map = gateset_opt_map + [
        [("rx90p[0]", drive.name, f"pwc{i}", "amp")],
        [("rx90p[0]", drive.name, f"pwc{i}", "t_final")],
        [("rx90p[0]", drive.name, f"pwc{i}", "freq_offset")],
        [("rx90p[0]", drive.name, f"pwc{i}", "xy_angle")],
        [("rx90p[0]", drive.name, f"pwc{i}", "delta")],
        [("rx90p[0]", drive.name, f"pwc{i}", "inphase")],
        [("rx90p[0]", drive.name, "carrier", "freq")],
        [("rx90p[0]", drive.name, "carrier", "framechange")],
    ]
print(gateset_opt_map)
parameter_map.set_opt_map(gateset_opt_map)

opt = OptimalControl(
    dir_path=output_dir + "/logs",
    fid_func=fidelities.unitary_infid_set,
    fid_subspace=[q.name for q in qubits],
    fid_func_kwargs={"active_levels": 4},
    pmap=parameter_map,
    algorithm=algorithms.lbfgs,
    options={"maxfun": 300},
    run_name="better_X90"
)
exp.set_opt_gates(opt_gates)
opt.set_callback(lambda fidelity: print(fidelity))
opt.set_exp(exp)
opt.optimize_controls()

[[('rx90p[0]', 'd1', 'pwc1', 'amp')], [('rx90p[0]', 'd1', 'pwc1', 't_final')], [('rx90p[0]', 'd1', 'pwc1', 'freq_offset')], [('rx90p[0]', 'd1', 'pwc1', 'xy_angle')], [('rx90p[0]', 'd1', 'pwc1', 'delta')], [('rx90p[0]', 'd1', 'pwc1', 'inphase')], [('rx90p[0]', 'd1', 'carrier', 'freq')], [('rx90p[0]', 'd1', 'carrier', 'framechange')], [('rx90p[0]', 'd2', 'pwc2', 'amp')], [('rx90p[0]', 'd2', 'pwc2', 't_final')], [('rx90p[0]', 'd2', 'pwc2', 'freq_offset')], [('rx90p[0]', 'd2', 'pwc2', 'xy_angle')], [('rx90p[0]', 'd2', 'pwc2', 'delta')], [('rx90p[0]', 'd2', 'pwc2', 'inphase')], [('rx90p[0]', 'd2', 'carrier', 'freq')], [('rx90p[0]', 'd2', 'carrier', 'framechange')]]
C3:STATUS:Saving as: /home/user/c3/output_1/logs/better_X90/2021_10_27_T_18_12_51/open_loop.log
0.912478560792998


/home/user/c3/c3/optimizers/optimizer.py:290: Warning: rx90p[0]-d1-pwc1-t_final has no gradient. This might indicate no usage for current experiment.
  warnings.warn(
/home/user/c3/c3/optimizers/optimizer.py:290: Warning: rx90p[0]-d2-pwc2-t_final has no gradient. This might indicate no usage for current experiment.
  warnings.warn(


0.9975973606275333
0.905283873223296
0.9999929933086653
0.9990577859254404
0.9422796733920706
0.967682842959218


In [19]:
# Print results
print(opt.current_best_goal)
for drive in drives:
    signal = generator.generate_signals(rx90p_q1)[drive.name]
    plotSignalWithEnvelope(signal["ts"].numpy(), signal["values"].numpy(),
                           getEnvelope(generator, rx90p_q1, drive.name),
                           pwc_times,
                           output_dir + f"/signal_after_{drive.name}.png", spectrum_cut=1e-4)

plot_dynamics(exp, init_state, sequence, output_dir + "/population_after.png")
population = runTimeEvolutionDefault(exp, init_state, sequence)
if len(qubits) > 1:
    plotSplittedOccupations(exp, population, sequence, output_dir + "/population_after_splitted.png")
plotMatrix(
    exp.propagators[rx90p_q1.get_key()],
    output_dir + "/propagator_after.png",
    output_dir + "/propagator_after_phase.png",
    output_dir + "/propagator_after_abs.png",
    labels=[str(x) for x in model.state_labels]
)
plotMatrix(
    exp.propagators[rx90p_q1.get_key()] - perfect_gate,
    output_dir + "/propagator_after_diff.png",
    output_dir + "/propagator_after_diff_phase.png",
    output_dir + "/propagator_after_diff_abs.png",
    labels=[str(x) for x in model.state_labels]
)
parameter_map.print_parameters()

0.0036165024711473093
saving plot in ./output_1/signal_after_d1.png
saving plot in ./output_1/propagator_after.png
saving plot in ./output_1/propagator_after_phase.png
saving plot in ./output_1/propagator_after_abs.png
saving plot in ./output_1/propagator_after_diff.png
saving plot in ./output_1/propagator_after_diff_phase.png
saving plot in ./output_1/propagator_after_diff_abs.png
rx90p[0]-d1-pwc-amp                   : 482.159 mV 
rx90p[0]-d1-pwc-t_final               : 15.000 ns 
rx90p[0]-d1-pwc-freq_offset           : -53.004 MHz 2pi 
rx90p[0]-d1-pwc-xy_angle              : -988.414 mrad 
rx90p[0]-d1-pwc-delta                 : -4.326  
rx90p[0]-d1-pwc-inphase               : -0.368  0.384  0.199  -0.115  -0.298  -0.247  -0.176  -0.133  -0.0933  -4.32e-05  0.177  0.262  0.201  -0.261  -0.861  -0.69  0.0964  0.335  0.307  0.0732  -0.143  -0.154  -0.095  -0.113  -0.209  -0.261  -0.0726  0.241  0.226  -0.303  
rx90p[0]-d1-carrier-freq              : 4.724 GHz 2pi 
rx90p[0]-d1-carrier-